In [137]:
import numpy as np
import pandas as pd
import datetime as dt

import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
from plotly.subplots import make_subplots

from src.forecasting.errors import mape

In [139]:
# read and initialize data for plots
date = dt.datetime.now().date()
time = dt.datetime.now().hour

results = pd.read_csv('data/forecasts/'+str(date)+'.csv',index_col=0,header=[0,1],date_parser=pd.to_datetime)
data = results.iloc[14:]

clusters = list(data.columns.get_level_values(0).unique())
models = list(data.columns.get_level_values(1).unique())
models.remove('actual')

# calculate mean absolute percentage errors for each forecast
mapes = {}
for cluster in clusters:
    mapes[cluster] = {}
    for model in models:
        mapes[cluster][model] = np.round(mape(data[cluster]['actual'],data[cluster][model]),3)

In [148]:
def LoadShapePlot(plotdata, **kwargs):
    """
    Takes a pandas.Series and returns a Plotly Scatter object with custom attributes
    kwargs are passed to plotly.graph_objs.Scatter
    """
    trace = go.Scatter(x=plotdata.index,y=plotdata.values,
                       mode='lines',
                       hoverinfo='x+y',
                       **kwargs)
    return trace

In [ ]:
# generate primary (aggregate load) figure
aggfig = go.Figure([LoadShapePlot(data['aggregate','actual'].iloc[:time]),
                    LoadShapePlot(data['aggregate','sarimax']),
                    LoadShapePlot(data['aggregate','mlp'])
                   ])
aggfig.update_layout(showlegend=False)



In [157]:

# initialize Dash app
app = dash.Dash()

styleDict = {
    'fontFamily':['Avenir','Arial'],
    'backgroundColor':'#FFFFFF',
    'color': '#111111',
    'textAlign':'center'
}

app.layout = html.Div(style=styleDict, children=[
    html.H1(
        children='University Load Forecasting:',
    ),
    html.Div(
        children="Applications of Deep Learning and Load-Shape Clustering"
    ),
    dcc.Graph(
        id='AggGraph',
        style={'width':800, 'height':500},
        figure=aggfig,
        config={
            # 'staticPlot': True,
            # 'modeBarButtons': [['pan2d','zoom2d']],
            'displaylogo': False,
            'showTips': False,
            'scrollZoom': False,
            'displayModeBar':False
        },
    ),
])

# if __name__ == '__main__':
#     app.server.run(port=8000, host='127.0.0.1', debug=True)
#     app.run_server(debug=True)